In [1]:
import numpy as np
import pandas as pd
import gurobipy as gb
from gurobipy import *

In [2]:
df = pd.read_csv('male_players (legacy).csv',encoding='latin1')
df['fifa_update_date'] = pd.to_datetime(df['fifa_update_date'])
df = df[df['nationality_name'] == 'Germany']
df = df[df['overall'] >= 79]

# remove injury player
df = df[df['player_id'] != 192318]

# Convert values in the 'preferred_foot' column to 0 for 'right' and 1 for 'left'
df['preferred_foot'] = df['preferred_foot'].apply(lambda x: 0 if x == 'Right' else 1)

# Choose 2018 
df = df[df['fifa_update_date'].dt.year == 2018]

/var/folders/3v/g2zdbcdx0czbtz7n9fzgd2180000gn/T/ipykernel_27255/13055136.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('male_players (legacy).csv',encoding='latin1')


In [3]:
name = df["short_name"].tolist()
pn = df.iloc[:, -28:-1].columns
names = [p+' : '+ item for item in name for p in pn]
foot = df['preferred_foot'].tolist()
overall = df["overall"].tolist()
num = len(name)
ability = df.iloc[:, -28:-1].values
for i in range(len(ability)):
    for j in range(len(ability[i])):
        ability[i][j] = eval(ability[i][j])
        if ability[i][j] not in range(0,100):
            print(ability[i][j])

In [4]:
model = gb.Model("best23")
#model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=names)
total = quicksum(n[i,j]*ability[i][j] for i in range(num) for j in range(27))
model.setObjective(total/23, GRB.MAXIMIZE)
for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==23)

# goal keeper equal to 3
model.addConstr(sum(n[i,26] for i in range(num))==3)

# forward at least 2
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))>=2)

# lb and rb at least 1 each
model.addConstr(sum(n[i,21] for i in range(num)) >=1)

model.addConstr(sum(n[i,25] for i in range(num)) >=1)

# rcb and lcb at least 2 each
model.addConstr(sum(n[i,22] for i in range(num)) >=2)
model.addConstr(sum(n[i,24] for i in range(num)) >=2)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= foot[i] for i in range(num))
model.addConstrs(n[i,16]<= foot[i] for i in range(num))

model.addConstrs(n[i,25] + foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + foot[i] <= 1 for i in range(num))

# Each position no more than 3 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=3 for j in range(27))

# Number of difference between left side and Right side should within 2 players
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l-r <=2)
model.addConstr(r-l <=2)

# Backfield at least 7 and no more than 9
back = sum(n[i,16] +n[i,20]+n[i,21]+n[i,22]+n[i,23]+n[i,24]+n[i,25]for i in range(num))
model.addConstr(back <=9)
model.addConstr(back >=7)

# Frontfield at least 3 and no more 5
f = sum(n[i,0] +n[i,1]+n[i,2]+n[i,3]+n[i,4]+n[i,5]+n[i,6]+n[i,7] for i in range(num))
model.addConstr(f <=5)
model.addConstr(f >=3)

Restricted license - for non-production use only - expires 2024-10-28


<gurobi.Constr *Awaiting Model Update*>

In [5]:
model.optimize()
for v in model.getVars():
       if v.x > 0:
            print(v.varName)
print('overall rating is: ',model.objVal)

gk : M. Neuer
lcm : T. Kroos
rcb : M. Hummels
gk : M. ter Stegen
ram : M. Özil
rcb : J. Boateng
cam : T. Müller
lw : L. Sané
rcm : S. Khedira
rm : M. Reus
rwb : J. Kimmich
ram : İ. Gündoğan
lcb : N. Süle
gk : T. Horn
lam : J. Draxler
lcm : L. Goretzka
st : T. Werner
lcb : J. Tah
rb : L. Bender
st : M. Kruse
lw : S. Gnabry
lam : J. Brandt
lb : J. Hector
overall rating is:  84.91304347826086


In [6]:
dfreal = pd.read_csv('male_players (legacy).csv',encoding='latin1')
dfreal['fifa_update_date'] = pd.to_datetime(dfreal['fifa_update_date'])
dfreal = dfreal[dfreal['fifa_update_date'].dt.year == 2018]
real_23player_list = [167495,182521,178603,192448,176635,183907,189596,179846,188350,212622,186942,212190,
                    202166,209658,212188,212194,150418,189251,205452,188943,207862,208334,192557]
real_df = dfreal[dfreal['player_id'].isin(real_23player_list)]
print(np.mean(real_df['overall']))
print(real_df[['player_id','short_name','overall']]) #84.45

83.91304347826087
       player_id       short_name  overall
68364     167495         M. Neuer       90
68366     182521         T. Kroos       90
68373     178603       M. Hummels       89
68375     192448    M. ter Stegen       89
68414     176635         M. Ãzil       86
68418     183907       J. Boateng       86
68423     189596       T. MÃ¼ller       86
68445     179846       S. Khedira       85
68451     188350          M. Reus       85
68461     212622       J. Kimmich       85
68486     186942   Ä°. GÃ¼ndoÄan       84
68507     212190         N. SÃ¼le       84
68538     202166       J. Draxler       83
68550     209658      L. Goretzka       83
68554     212188        T. Werner       83
68651     212194        J. Brandt       82
68676     150418        M. GÃ³mez       81
68706     189251          S. Rudy       81
68744     205452      A. RÃ¼diger       81
68813     188943         K. Trapp       80
68863     207862        M. Ginter       80
69014     208334        J. Hector   

/var/folders/3v/g2zdbcdx0czbtz7n9fzgd2180000gn/T/ipykernel_27255/2483571341.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  dfreal = pd.read_csv('male_players (legacy).csv',encoding='latin1')


In [7]:
# real 23 squad and rating
real_11player_list = [167495,182521,208334,205452,183907,212622,189251,188350,202166,189596,212188]
real_11df = dfreal[dfreal['player_id'].isin(real_11player_list)]
print(np.mean(real_11df['overall']))

84.45454545454545


In [8]:
# real team 11 line up overall rating
print((90+85+80+81+86+79+85+85+85+86+83)/11)

84.0909090909091


In [9]:
new_23_ability = []
new_23_name = []
new_foot = []
for i in range(num):
    if sum(n[i,j].x for j in range(27)) ==1:
        new_23_ability.append(ability[i].tolist())
        new_23_name.append(name[i])
        new_foot.append(foot[i])
num = len(new_23_name)
new_23_name = [p+' : '+ item for item in new_23_name for p in pn]

In [10]:
# No prior formation
model = gb.Model("best11")
#model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=new_23_name)
total = quicksum(n[i,j]*new_23_ability[i][j] for i in range(num) for j in range(27))
model.setObjective(total/11, GRB.MAXIMIZE)
for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==11)

# goal keeper equal to 1
model.addConstr(sum(n[i,26] for i in range(num))==1)

# forward at least 1
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))>=1)

# lb,lwb and rb,rwb only 1 each
model.addConstr(sum(n[i,21]+n[i,16] for i in range(num)) ==1)

model.addConstr(sum(n[i,25]+n[i,20] for i in range(num)) ==1)

# if has lwb then must have rwb and must have one CB
model.addConstr(sum(n[i,16] for i in range(num)) == sum(n[i,20] for i in range(num)))
model.addConstr(sum(n[i,16] for i in range(num)) == sum(n[i,23] for i in range(num)))

# if has lb then must have rb
model.addConstr(sum(n[i,25] for i in range(num)) == sum(n[i,21] for i in range(num)))

# rcb and lcb only 1 each
model.addConstr(sum(n[i,22] for i in range(num)) ==1)
model.addConstr(sum(n[i,24] for i in range(num)) ==1)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= new_foot[i] for i in range(num))
model.addConstrs(n[i,16]<= new_foot[i] for i in range(num))

model.addConstrs(n[i,25] + new_foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + new_foot[i] <= 1 for i in range(num))

# Each position only can be 1 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=1 for j in range(27))

# Number of left side and Right side should be equal
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l==r)

rf = sum(n[i,6]+n[i,7]+n[i,15] for i in range(num))
lf = sum(n[i,3]+n[i,4]+n[i,11] for i in range(num))
model.addConstr(lf==rf)

# Midfield at least 2
f = sum(n[i,12] +n[i,13]+n[i,14]+n[i,17]+n[i,18]+n[i,19] for i in range(num))
model.addConstr(f >=2)

<gurobi.Constr *Awaiting Model Update*>

In [11]:
model.optimize()
for v in model.getVars():
       if v.x > 0:
            print(v.varName)
print('overall rating is: ',model.objVal)

gk : M. Neuer
rcm : T. Kroos
lcb : M. Hummels
lam : M. Özil
rcb : J. Boateng
st : T. Müller
rm : L. Sané
cdm : S. Khedira
lm : M. Reus
rb : J. Kimmich
lb : J. Hector
overall rating is:  86.09090909090911


In [12]:
#3 back formation
model = gb.Model("3back best11")
#model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=new_23_name)
total = quicksum(n[i,j]*new_23_ability[i][j] for i in range(num) for j in range(27))
model.setObjective(total/11, GRB.MAXIMIZE)
for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==11)

# goal keeper equal to 1
model.addConstr(sum(n[i,26] for i in range(num))==1)

# forward at least 1
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))>=1)

# no lb and rb 1 each
model.addConstr(sum(n[i,21] for i in range(num)) ==0)

model.addConstr(sum(n[i,25] for i in range(num)) ==0)

# lb,lwb and rb,rwb only 1 each
model.addConstr(sum(n[i,21]+n[i,16] for i in range(num)) ==1)

model.addConstr(sum(n[i,25]+n[i,20] for i in range(num)) ==1)

# one CB
model.addConstr(sum(n[i,23] for i in range(num)) == 1)

# rcb and lcb only 1 each
model.addConstr(sum(n[i,22] for i in range(num)) ==1)
model.addConstr(sum(n[i,24] for i in range(num)) ==1)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= new_foot[i] for i in range(num))
model.addConstrs(n[i,16]<= new_foot[i] for i in range(num))

model.addConstrs(n[i,25] + new_foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + new_foot[i] <= 1 for i in range(num))

# Each position only can be 1 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=1 for j in range(27))

# Number of left side and Right side should be equal
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l==r)

rf = sum(n[i,6]+n[i,7]+n[i,15] for i in range(num))
lf = sum(n[i,3]+n[i,4]+n[i,11] for i in range(num))
model.addConstr(lf==rf)


# Midfield at least 2
f = sum(n[i,12] +n[i,13]+n[i,14]+n[i,17]+n[i,18]+n[i,19] for i in range(num))
model.addConstr(f >=2)

<gurobi.Constr *Awaiting Model Update*>

In [13]:
model.optimize()
for v in model.getVars():
       if v.x > 0:
            print(v.varName)
print('overall rating is: ',model.objVal)

gk : M. Neuer
cm : T. Kroos
cb : M. Hummels
ram : M. Özil
lcb : J. Boateng
st : T. Müller
lcm : S. Khedira
cam : M. Reus
rwb : J. Kimmich
rcb : N. Süle
lwb : J. Hector
overall rating is:  86.00000000000001
